**Note:** Make sure you go through the [Setup Notebook](penguins-setup.ipynb) notebook once at the start of the program.

In [ ]:
# !pip install -q --upgrade pip
# !pip install -q --upgrade awscli boto3
!pip install -q --upgrade scikit-learn==0.23.2
# !pip install -q --upgrade PyYAML==6.0
!pip install -q --upgrade pip
!pip install -q --upgrade awscli boto3
!pip install -q --upgrade sagemaker==2.173.0
!pip install -q --upgrade sagemaker_inference
!pip show sagemaker

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

CODE_FOLDER = Path("code")
ENDPOINT_CODE_FOLDER = Path("code/endpoint")
sys.path.append(f"./{CODE_FOLDER}")

In [ ]:
import json
import os
import random
import tarfile
import tempfile
import time
from datetime import datetime
from threading import Event, Thread
from time import sleep

import numpy as np
from constants import *
from endpoint.inference import *
from evaluation import evaluate
from IPython.display import JSON
from preprocessor import preprocess
from sagemaker import ModelPackage
from sagemaker.deserializers import JSONDeserializer
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.inputs import (CreateModelInput, FileSystemInput, TrainingInput,
                              TransformInput)
from sagemaker.lambda_helper import Lambda
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.model_monitor import (CronExpressionGenerator,
                                     DefaultModelMonitor, EndpointInput,
                                     ModelQualityMonitor, MonitoringExecution)
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.parameter import ContinuousParameter, IntegerParameter
from sagemaker.predictor import Predictor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.pytorch import PyTorch, PyTorchProcessor
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.serializers import JSONSerializer
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import HyperparameterTuner
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import (ConditionGreaterThanOrEqualTo,
                                           ConditionLessThanOrEqualTo)
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.workflow.lambda_step import (LambdaOutput, LambdaOutputTypeEnum,
                                            LambdaStep)
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.parameters import (ParameterBoolean, ParameterFloat,
                                           ParameterInteger, ParameterString)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline_definition_config import \
    PipelineDefinitionConfig
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.quality_check_step import (DataQualityCheckConfig,
                                                   ModelQualityCheckConfig,
                                                   QualityCheckStep)
from sagemaker.workflow.steps import (CacheConfig, CreateModelStep,
                                      ProcessingStep, TrainingStep,
                                      TransformStep, TuningStep)


from constants import *
from nn import train

MODEL_PACKAGE_GROUP = "penguins"
ENDPOINT = "penguins-endpoint"

In [ ]:
# Test preprocessing script
with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    print(f"Folders: {os.listdir(directory)}")

In [ ]:
# preprocess_data_step parameters
dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data.csv",
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f"{S3_LOCATION}/preprocessing",
)

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

In [ ]:
# Define preprocessing step
preprocess_data_step = ProcessingStep(
    name="preprocess-data",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
        ProcessingOutput(output_name="classes", source="/opt/ml/processing/classes", destination=preprocessor_destination),
        ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline"),
        ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline"),
    ],
    code=f"{CODE_FOLDER}/preprocessor.py",
    cache_config=cache_config
)



In [ ]:
# Undesirable but this is a workaround until a means can be found in importing libs for the deploy_step
!cp {CODE_FOLDER}/nn.py  {ENDPOINT_CODE_FOLDER}/nn.py
!cp {CODE_FOLDER}/constants.py  {ENDPOINT_CODE_FOLDER}/constants.py

In [ ]:

# Create a temporary directory to test the training.
with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=100,
        learning_rate=0.01
    )

In [ ]:
# Define tuning step
objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 100),
    "batch_size": IntegerParameter(8, 16),
    "learning_rate": ContinuousParameter(0.007, 0.01),
}

estimator = PyTorch(
    entry_point=f"{CODE_FOLDER}/nn.py",
    framework_version="1.8",
    instance_type="ml.m5.large",
    py_version="py36",
    instance_count=1,
    script_mode=True,
    
    # The default profiler rule includes a timestamp which will change each time
    # the pipeline is upserted, causing cache misses. Since we don't need
    # profiling, we can disable it to take advantage of caching.
    disable_profiler=True,

    role=role,
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

In [ ]:
tune_model_step = TuningStep(
    name = "tune-model",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

In [ ]:
with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
        evaluation_name="evaluation",
    )
    
    with open(Path(directory) / "evaluation" / f"evaluation.json", "r") as file:
        data = json.load(file)
        print(data)

In [ ]:
pytorch_processor = PyTorchProcessor(
    base_job_name="penguins-evaluation-processor",
    framework_version="1.8",
    py_version="py36",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
    sagemaker_session=PipelineSession(),
)

# This is a workaround to a problem with the SageMaker SDK: 
# By default, the TensorFlowProcessor runs the script using
# /bin/bash as its entrypoint. We want to ensure we run it 
# using python3.
pytorch_processor.framework_entrypoint_command = ["python3"]

eval_winner_name = "evaluate-winner-model"

# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
def create_eval_report(report_name):
    return PropertyFile(
        name=report_name,
        output_name="evaluation",
        path=f"{report_name}.json",
    )

def create_eval_process_step(evaluation_name, report, top_k=0):
    return ProcessingStep(
        name=evaluation_name,
        processor=pytorch_processor,
        inputs=[
            ProcessingInput(source=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test"
            ),
            ProcessingInput(
                source=(
                    tune_model_step.get_top_model_s3_uri(top_k=top_k, s3_bucket=sagemaker_session.default_bucket()) 
                ),
                destination="/opt/ml/processing/model",
            )
        ],
        outputs=[
            ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=f"{S3_LOCATION}/evaluation"),
        ],
        code=f'{CODE_FOLDER}/evaluation.py',
        job_arguments=["--evaluation_name", evaluation_name],
        property_files=[report],
        cache_config=cache_config,
    )
    
eval_winner_report = create_eval_report(eval_winner_name)

eval_model_step = create_eval_process_step(eval_winner_name, 
                                        eval_winner_report)


In [ ]:
evaluation_model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="/", values=[
            eval_model_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'], f"{eval_winner_name}.json"]
        ),
        content_type="application/json",
    )
)

model_package_group_name = "penguins"

In [ ]:
def get_model(top_k=0):
    return PyTorchModel(
        model_data=(
            tune_model_step.get_top_model_s3_uri(top_k, s3_bucket=PipelineSession().default_bucket())
        ),
        framework_version="1.8",
        py_version="py36",
        sagemaker_session=PipelineSession(),
        role=role,
    )

def model_registry_args(model,
                        model_metrics, 
                        approval_status="PendingManualApproval"):
    return model.register(
        model_package_group_name=model_package_group_name,
        model_metrics=evaluation_model_metrics,
        approval_status=approval_status,
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )

model = get_model() 

args_approved = model_registry_args(model, 
                                    evaluation_model_metrics, 
                                    "Approved")

args_pending = model_registry_args(model,
                                evaluation_model_metrics)

register_step_approved = ModelStep(
    name="register-model-approved",
    step_args=args_approved,
)

register_step_pending = ModelStep(
    name="register-model-pending-approval",
    step_args=args_pending,
)


In [ ]:
accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.70
)

min_accuracy_threshold = ParameterFloat(
    name="accuracy_threshold_min", 
    default_value=0.50
)

def gte_eval_condition(step, report, accuracy):
    return ConditionGreaterThanOrEqualTo(
        left=JsonGet(
            step_name=step.name,
            property_file=report,
            json_path="metrics.accuracy.value"
        ),
        right=accuracy
    )

def lte_eval_condition(step, report, accuracy):
    return ConditionLessThanOrEqualTo(
        left=JsonGet(
            step_name=step.name,
            property_file=report,
            json_path="metrics.accuracy.value"
        ),
        right=accuracy
    )


gte_approved = gte_eval_condition(eval_model_step, 
                                eval_winner_report,
                                accuracy_threshold)

lte_min = lte_eval_condition(eval_model_step,
                            eval_winner_report,
                            min_accuracy_threshold)


gte_min = gte_eval_condition(eval_model_step,
                            eval_winner_report,
                            min_accuracy_threshold)

lte_approved = lte_eval_condition(eval_model_step,
                               eval_winner_report,
                               accuracy_threshold)

fail_step_min = FailStep(
    name="fail-min",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            min_accuracy_threshold
        ]
    ),
)

def create_condition_step(name, conditions, if_steps, else_steps=None):
    return ConditionStep(
        name=name,
        conditions=conditions,
        if_steps=if_steps,
        else_steps=else_steps
    )

check_min_step = create_condition_step("min-model-accuracy",
                                        [lte_min],
                                        [fail_step_min],)

approved_step = create_condition_step("approved-model-accuracy", 
                                        [gte_approved],
                                        [register_step_approved])
pending_step = create_condition_step("pending-model-accuracy",
                                        [lte_approved, gte_min],
                                        [register_step_pending],)

In [ ]:
ENDPOINT_CODE_FOLDER = CODE_FOLDER / "endpoint"
Path(ENDPOINT_CODE_FOLDER).mkdir(parents=True, exist_ok=True)
sys.path.append(f"./{ENDPOINT_CODE_FOLDER}")

We will include the inference code as part of the model assets to control the inference process on the SageMaker endpoint. SageMaker will automatically call the `handler()` function for every request to the endpoint.

In [ ]:
samples = {
    "island": "Biscoe",
    "culmen_length_mm": 18.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 1800.0,
    "species": "groundtruth"
}

with tempfile.TemporaryDirectory() as directory:
    
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )
    
    request_content_type = "application/json"
    
    model = model_fn(Path(directory) / "model")
    transformed_list = input_fn(json.dumps(samples), request_content_type, directory)
    output_np = predict_fn(transformed_list, model)
    predictions = output_fn(output_np,directory)
    print(predictions)
    

SageMaker's default TensorFlow inference container doesn't come with Scikit-Learn installed, so we need to provide a `requirements.txt` file with the libraries we want SageMaker to install in our endpoint.

In [ ]:
%%writefile {ENDPOINT_CODE_FOLDER}/requirements.txt

numpy==1.19.5
pandas==1.1.5
scikit-learn==0.23.2
boto3
sagemaker_inference

In [ ]:
repacked_model = PyTorchModel(
    name="penguins",
    model_data=(
        tune_model_step.get_top_model_s3_uri(top_k=0, s3_bucket=PipelineSession().default_bucket())
    ),
    framework_version="1.8",
    py_version="py36",
    sagemaker_session=PipelineSession(),
    role=role,
    entry_point="inference.py",
    source_dir=str(ENDPOINT_CODE_FOLDER),
    env={
        "PIPELINE_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["pipeline"].S3Output.S3Uri,
                "pipeline.pkl",
            ]
        ),
        "CLASSES_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["classes"].S3Output.S3Uri,
                "classes.csv",
            ]
        )
    }    
)

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=repacked_model.register(
        model_package_group_name=model_package_group_name,
        approval_status="Approved",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )

)

In [ ]:
try:
    response = boto3.client("sagemaker").delete_endpoint(
        EndpointName="penguins-endpoint",
    )
    
    response = boto3.client("sagemaker").delete_endpoint_config(
        EndpointConfigName='penguins-endpoint'
    )
except:
    print("Endpoint does not exist")
    


In [ ]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = response["ModelPackageSummaryList"][0] if response["ModelPackageSummaryList"] else None
package

model_package = ModelPackage(
    model_package_arn=package["ModelPackageArn"], 
    sagemaker_session=sagemaker_session,
    role=role, 
)

In [ ]:
%%writefile {CODE_FOLDER}/lambda.py

import os
import json
import boto3
import time

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_package_arn = event["model_package_arn"]
    endpoint_name = event["endpoint_name"]
    data_capture_percentage = event["data_capture_percentage"]
    data_capture_destination = event["data_capture_destination"]
    role = event["role"]
    timestamp = time.strftime("%m%d%H%M%S", time.localtime())
    model_name = f"penguins-model-{timestamp}"
    endpoint_config_name = f"penguins-endpoint-config-{timestamp}"

    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": model_package_arn
        }] 
    )

    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ],
        DataCaptureConfig={
            "EnableCapture": True,
            "InitialSamplingPercentage": data_capture_percentage,
            "DestinationS3Uri": data_capture_destination,
            "CaptureOptions": [
                {
                    'CaptureMode': "Input"
                },
                {
                    'CaptureMode': "Output"
                },
            ],
            "CaptureContentTypeHeader": {
                "JsonContentTypes": [
                    "application/json",
                    "application/octect-stream"
                ]
            }
        },
    )

    sagemaker.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully")
    }

In [ ]:
def create_lambda_role(role_name):
    try:
        response = iam_client.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description="Lambda Pipeline Role"
        )

        role_arn = response['Role']['Arn']

        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        iam_client.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )

        return role_arn

    except iam_client.exceptions.EntityAlreadyExistsException:
        response = iam_client.get_role(RoleName=role_name)
        return response['Role']['Arn']


lambda_role = create_lambda_role("lambda-pipeline-role")

In [ ]:
data_capture_percentage = ParameterInteger(
    name="data_capture_percentage",
    default_value=100,
)

data_capture_destination = ParameterString(
    name="data_capture_destination",
    default_value=f"{S3_LOCATION}/monitoring/data-capture",
)

deploy_fn = Lambda(
    function_name="deploy_fn",
    execution_role_arn=lambda_role,
    script=str(CODE_FOLDER / "lambda.py"),
    handler="lambda.lambda_handler",
    timeout=600
)

deploy_fn.upsert()

deploy_step = LambdaStep(
    name="deploy",
    lambda_func=deploy_fn,
    inputs={
        # We use the ARN of the model we registered to
        # deploy it to the endpoint.
        "model_package_arn": register_model_step.properties.ModelPackageArn,

        "endpoint_name": "penguins-endpoint",
        
        "data_capture_percentage": data_capture_percentage,
        "data_capture_destination": data_capture_destination,
        
        "role": role,
    }
)

In [ ]:
DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"

In [ ]:
files = S3Downloader.list(data_capture_destination.default_value)[:3]
files

In [ ]:
if len(files):
    lines = S3Downloader.read_file(files[0])
    print(json.dumps(json.loads(lines.split("\n")[0]), indent=2))

In [ ]:
data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=sagemaker_session,
        role=role,
    ),
    
    quality_check_config = DataQualityCheckConfig(
        # We will use the train dataset we generated during the preprocessing 
        # step to generate the data quality baseline.
        baseline_dataset=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,

        dataset_format=DatasetFormat.json(lines=True),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=model_package_group_name,
    cache_config=cache_config
)

In [ ]:
DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"
MODEL_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/model-quality"

files = S3Downloader.list(data_capture_destination.default_value)[:3]
files

if len(files):
    lines = S3Downloader.read_file(files[0])
    print(json.dumps(json.loads(lines.split("\n")[0]), indent=2))
    
data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.large",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=PipelineSession(),
        role=role,
    ),
    
    quality_check_config = DataQualityCheckConfig(
        # We will use the train dataset we generated during the preprocessing 
        # step to generate the data quality baseline.
        baseline_dataset=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,

        dataset_format=DatasetFormat.json(lines=True),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=MODEL_PACKAGE_GROUP,
    cache_config=cache_config
)

create_model_step = ModelStep(
    name="create",
    display_name="create-model",
    step_args=repacked_model.create(
        instance_type="ml.m5.large"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    base_transform_job_name="transform",

    instance_type="ml.m5.large",
    instance_count=1,
    
    accept="application/json",
    strategy="SingleRecord",
    assemble_with="Line",
    
    output_path=f"{S3_LOCATION}/transform",
    sagemaker_session=PipelineSession()
)

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    step_args=transformer.transform(
        # We will use the test dataset we generated during the preprocessing 
        # step to run it through the model and generate predictions.
        data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["test-baseline"].S3Output.S3Uri,

        join_source="Input",
        content_type="application/json",
        split_type="Line",
        output_filter="$.SageMakerOutput['prediction','groundtruth']",
    ),
    cache_config=cache_config
)

model_quality_location = f"{S3_LOCATION}/monitoring/model-quality"

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=PipelineSession(),
        role=role,
    ),
    
    quality_check_config = ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,

        dataset_format=DatasetFormat.json(lines=True),

        # As a work around a bug in QualityCheckStep when using JSONPath(s).
        # We counter-intuitively send the groundtruth from test data to calculate baseline performance.         
        # https://github.com/aws/sagemaker-python-sdk/issues/4130
        problem_type="MulticlassClassification",
        inference_attribute="species",
        ground_truth_attribute="species",

        output_s3_uri=model_quality_location,
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=MODEL_PACKAGE_GROUP,
    cache_config=cache_config
)

model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=repacked_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="Approved",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )
)

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[gte_approved],
    if_steps=[
        create_model_step, 
        generate_test_predictions_step, 
        model_quality_baseline_step, 
        register_model_step,
        deploy_step
    ],
    else_steps=[fail_step_min], 
)


In [ ]:
create_model_step = ModelStep(
    name="create",
    display_name="create-model",
    step_args=repacked_model.create(
        instance_type="ml.m5.large"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    base_transform_job_name="transform",

    instance_type="ml.m5.large",
    instance_count=1,
    
    accept="application/json",
    strategy="SingleRecord",
    assemble_with="Line",
    
    output_path=f"{S3_LOCATION}/transform",
)

# Workaround for bug in SDK version 2.171.0
# https://github.com/aws/sagemaker-python-sdk/issues/3991
transformer._current_job_name = "transform"

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    transformer=transformer,
    inputs=TransformInput(
        
        # We will use the test dataset we generated during the preprocessing 
        # step to run it through the model and generate predictions.
        data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["test-baseline"].S3Output.S3Uri,

        join_source="Input",
        content_type="application/json",
        split_type="Line",
    ),
    cache_config=cache_config
)

In [ ]:
register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=repacked_model.register(
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        drift_check_baselines=drift_check_baselines,
        approval_status="Approved",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )
)


In [ ]:
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[gte_approved],
    if_steps=[
        create_model_step, 
        generate_test_predictions_step,
        model_quality_baseline_step,
        register_model_step,
        deploy_step
    ],
    else_steps=[fail_step_min], 
)

In [ ]:
pipeline = Pipeline(
    name="final-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        data_capture_percentage,
        data_capture_destination,
        accuracy_threshold,
        min_accuracy_threshold,
    ],
    steps=[
        preprocess_data_step,
        data_quality_baseline_step,
        tune_model_step,
        eval_model_step,
        condition_step,         
    ],
    pipeline_definition_config=pipeline_definition_config
)

pipeline.upsert(role_arn=role)

# Delete endpoint(if already exists) as Pipeline will fail on the Lambda step(deployment)
try:
    response = boto3.client("sagemaker").delete_endpoint(
        EndpointName="penguins-endpoint",
    )
except:
    print("Endpoint does not exist")
    
pipeline.start()

In [ ]:

endpoint = 'penguins-endpoint'

# Read image into memory
payload=[{
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    },{
        "island": "Dream",
        "culmen_length_mm": 43.2,
        "culmen_depth_mm": 17.5,
        "flipper_length_mm": 175.0,
        "body_mass_g": 3500.0,
    },{
        "island": "Torgersen",
        "culmen_length_mm": 38.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 176.0,
        "body_mass_g": 3700.0,
    }]

predictor = Predictor("penguins-endpoint")
inference_response = predictor.predict(data=json.dumps(payload))
print (inference_response)